In [1]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
#labelencoding을 위한 패키지 import 


In [3]:
data = pd.read_csv('train_data.csv', encoding='euc-kr')
display(data)
display(data.columns)

,날짜,요일,홈,어웨이,관중수,홈팀 순위,홈팀승률,홈팀연승,홈 연승 숫자로,어웨이 순위,...,어웨이 연승,어웨이 연승 숫자로,평균기온,합계 일사량(MJ/m2),일강수량(mm),공휴일 및 주말,이벤트 여부,경기 시간,관중수.1,비율(y)
0,2022-04-02,토,두산,한화,"16,271",1,1.000,1승,1,6,...,1패,-1,8.2,24.09,NaN,1,NaN,0,"16,271",0.685
1,2022-04-03,일,두산,한화,"11,345",1,1.000,2승,2,8,...,2패,-2,9.6,24.34,NaN,1,1.0,0,"11,345",0.478
2,2022-04-05,화,두산,삼성,"4,109",3,0.667,2승,2,3,...,2승,2,10.3,21.21,NaN,0,NaN,1,"4,109",0.173
3,2022-04-06,수,두산,삼성,"4,161",4,0.500,1패,-1,3,...,3승,3,11.7,21.54,NaN,0,NaN,1,"4,161",0.175
4,2022-04-07,목,두산,삼성,"4,055",3,0.600,1승,1,3,...,1패,-1,10.9,24.30,NaN,0,NaN,1,"4,055",0.171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,2023-10-12,목,두산,NC,"7,113",5,0.526,1승,1,4,...,1패,-1,17.5,14.40,NaN,0,NaN,1,"7,113",0.299
284,2023-10-13,금,두산,KIA,"19,188",5,0.529,2승,2,6,...,1패,-1,17.5,9.14,NaN,0,1.0,1,"19,188",0.808
285,2023-10-14,토,LG,두산,"21,771",1,0.603,2패,-2,3,...,3승,3,14.7,6.54,6.0,1,NaN,0,"21,771",0.917
286,2023-10-15,일,LG,두산,"23,750",1,0.606,1승,1,5,...,1패,-1,16.1,13.73,NaN,1,1.0,0,"23,750",1.000


Index(['날짜', '요일', '홈', '어웨이', '관중수', '홈팀 순위', '홈팀승률', '홈팀연승', '홈 연승 숫자로',
       '어웨이 순위', '어웨이 승률', '어웨이 연승', '어웨이 연승 숫자로', '평균기온', '합계 일사량(MJ/m2)',
       '일강수량(mm)', '공휴일 및 주말', '이벤트 여부', '경기 시간', '관중수.1', '비율(y)'],
      dtype='object')

In [8]:
#one-hot encoding 

import pandas as pd

# CSV 파일 읽기
file_path = 'test_data.csv'
data = pd.read_csv(file_path, encoding='utf-8')

# 'Unnamed'로 시작하는 컬럼 제거
data = data.drop([col for col in data.columns if 'Unnamed' in col], axis=1)


# '날짜' 열이 있는지 확인 후 전처리 수행
if '날짜' in data.columns:
    data['날짜'] = pd.to_datetime(data['날짜'])

    # 년과 월로 구분하여 인덱싱
    data['년'] = data['날짜'].dt.year
    data['월'] = data['날짜'].dt.month

    # 요일을 숫자로 변환
    day_mapping = {'월': 0, '화': 1, '수': 2, '목': 3, '금': 4, '토': 5, '일': 6}
    data['요일'] = data['요일'].map(day_mapping)

    # 결측치 처리
    data = data.fillna(0)

    # 홈과 어웨이를 숫자로 변환
    encoder_home = LabelEncoder()  #카테고리 값을 숫자로 변환 
    data['홈_레이블'] = encoder_home.fit_transform(data['홈']) 
    #홈열으 ㅣ각 카테고리 값을 고유한 정수로 매핑하고 '홈_레이블'에 저장
    encoder_away = LabelEncoder()  #카테고리 값을 숫자로 변환 
    encoder_away.fit(data['어웨이']) #고유의 카테고리값을 학습

    labels_away = encoder_away.transform(encoder_away.classes_) #어웨이 열의 모든 고유 카테고리 값을 숫자로 변환
    away_mapping = dict(zip(encoder_away.classes_, labels_away)) #각 카테고리오 ㅏ숫자 간의 매핑을 dict을 ㅗ저장

    #특정 팀에게 수동으로 우선 순위를 부여
    priority_teams = {'LG': 0, '두산': 1}
    
    #사전의 각 팀에 대해 해당 팀이 'away_mapping' dict에 있으면 해당 팁의 값을 'priority_teams' 값으로 갱신
    for team in priority_teams:
        if team in away_mapping:
            away_mapping[team] = priority_teams[team]

    #변수 초기값 2로 설정. dict에 있는 팀을 순회하면서 'priority_teams'에 없으면 새로운 라벨 부여 
    #각 팀에 대해 new_label 할당하고 증가 

    new_label = 2
    for team in away_mapping:
        if team not in priority_teams:
            away_mapping[team] = new_label
            new_label += 1

    #어웨이 열을 새로운 라벨 값으로 매핑 
    data['어웨이_레이블'] = data['어웨이'].map(away_mapping)

    # 필요 없는 컬럼 제거
    data = data.drop(['홈','관중수','어웨이', '홈팀연승', '어웨이 연승', '날짜'], axis=1)

    # 컬럼명 변경
    data = data.rename(columns={
        '요일': 'day', '홈팀 순위':'home_rank', '어웨이 순위':'away_rank', '홈 연승 숫자로':'home_winning', 
        '어웨이 연승 숫자로':'away_winning', '홈_레이블':'home', '어웨이_레이블':'away', '비율(y)':'target', 
        '년':'year', '월':'month', '일강수량(mm)': 'rain', '평균기온': 'temp',  '합계 일사량(MJ/m2)': 'insolation', 
        '경기 시간':'time', '공휴일 및 주말':'holiday','이벤트 여부':'event','홈팀승률':'home_winning_rate', 
        '어웨이 승률':'away_winning_rate'
    })

    #one-hot encoding을 위한 get_dummies 진행(요일과 팀명 구분)
    data = pd.get_dummies(data, columns=['day'])
    data = pd.get_dummies(data, columns=['home', 'away'])
    data= data.astype(int)
    data.columns

    #away와 home을 통일하기 위한 컬럼 합치기 
    data['away_0'] = data['away_0'].astype(int) + data['home_0'].astype(int)
    data['away_1'] = data['away_1'].astype(int) + data['home_1'].astype(int)

    
    #home컬럼 제거
    #lg가 홈이고 두산이 원정인 경우와 
    #두산이 홈이고 lg가 원정인 경우는
    #결국 두팀이 잠실에서 한다는 것으로 같기 때문에
    #그냥 경기하는 팀만 one-hot encoding으로 남김

    data = data.drop(columns=['home_0', 'home_1'])

    #rename
    data = data.rename(columns={'away_0': 'LG', 'away_1': 'DOOSAN','away_2': 'KIA','away_3': 'KT','away_4': 'NC','away_5': 'SSG','away_6': 'LOTTE','away_7': 'SAMSUNG','away_8': 'KIWOOM','away_9': 'HANHWA', 'day_0': 'MON', 'day_1': 'TUE', 'day_2': 'WEN', 'day_3': 'THUR', 'day_4': 'FRI', 'day_5': 'SAT', 'day_6': 'SUN'})
    # 데이터 표시 테스트 셋에는 월요일에 경기가 벌어진 것이 없기 때문에 컬럼을 맞춰주기 위해 삽입
    data['MON'] = 0
    display(data.columns, len(data.columns))
    display(data)
else:
    print("Error: '날짜' 열이 데이터에 존재하지 않습니다.")

data = data.to_csv('test_final_data_2.csv', index=False, encoding='euc-kr')

Index(['home_rank', 'home_winning_rate', 'home_winning', 'away_rank',
       'away_winning_rate', 'away_winning', 'temp', 'insolation', 'rain',
       'holiday', 'event', 'time', 'target', 'year', 'month', 'TUE', 'WEN',
       'THUR', 'FRI', 'SAT', 'SUN', 'LG', 'DOOSAN', 'KIA', 'KT', 'NC', 'SSG',
       'LOTTE', 'SAMSUNG', 'KIWOOM', 'HANHWA', 'MON'],
      dtype='object')

32

,home_rank,home_winning_rate,home_winning,away_rank,away_winning_rate,away_winning,temp,insolation,rain,holiday,...,DOOSAN,KIA,KT,NC,SSG,LOTTE,SAMSUNG,KIWOOM,HANHWA,MON
0,1,1,1,6,0,-1,13,19,0,1,...,0,0,0,0,0,0,0,0,1,0
1,4,0,-1,4,0,1,16,19,0,1,...,0,0,0,0,0,0,0,0,1,0
2,2,0,1,2,0,-1,7,13,11,0,...,0,0,0,0,0,0,1,0,0,0
3,5,0,1,5,0,-1,10,18,0,0,...,0,0,0,0,0,0,1,0,0,0
4,3,0,2,6,0,-2,10,4,1,0,...,0,0,0,0,0,0,1,0,0,0
5,5,0,-2,1,1,4,7,6,1,0,...,1,1,0,0,0,0,0,0,0,0
6,5,0,1,2,0,-1,8,16,0,1,...,1,1,0,0,0,0,0,0,0,0
7,6,0,-1,2,0,1,10,22,0,1,...,1,1,0,0,0,0,0,0,0,0
8,5,0,-3,2,0,3,15,21,0,0,...,0,0,0,1,0,0,0,0,0,0
9,5,0,1,4,0,-1,16,7,0,0,...,0,0,0,1,0,0,0,0,0,0
